# Ajouts de Features 

In [1]:
import pandas as pd
import numpy as np

In [3]:
MNCAATourneyDetailedResults = pd.read_csv('Data/MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')
MNCAATourneyDetailedResults = MNCAATourneyDetailedResults.loc[MNCAATourneyDetailedResults['Season'] >= 2003]
MNCAATourneyDetailedResults = MNCAATourneyDetailedResults.loc[MNCAATourneyDetailedResults['Season'] <= 2014]
MNCAATourneyDetailedResults = MNCAATourneyDetailedResults[['Season','DayNum','WTeamID','LTeamID']]
MNCAATourneyDetailedResults.loc[:,'Competition'] = 'MNCAA'
MNCAATourneyDetailedResults.loc[:,'WTeam'] = 0

for i,_ in enumerate(MNCAATourneyDetailedResults.iterrows()):
    if MNCAATourneyDetailedResults.iloc[i,2] > MNCAATourneyDetailedResults.iloc[i,3] :
        MNCAATourneyDetailedResults.iloc[i,2],MNCAATourneyDetailedResults.iloc[i,3] = MNCAATourneyDetailedResults.iloc[i,3],MNCAATourneyDetailedResults.iloc[i,2]
        MNCAATourneyDetailedResults.iloc[i,5] = 1
        
MNCAATourneyDetailedResults = MNCAATourneyDetailedResults.rename(columns={"WTeamID": "Team0_ID", "LTeamID": "Team1_ID"})

MNCAATourneyDetailedResults

,Season,DayNum,Team0_ID,Team1_ID,Competition,WTeam
0,2003,134,1411,1421,MNCAA,1
1,2003,136,1112,1436,MNCAA,0
2,2003,136,1113,1272,MNCAA,0
3,2003,136,1141,1166,MNCAA,0
4,2003,136,1143,1301,MNCAA,0
...,...,...,...,...,...,...
775,2014,146,1163,1277,MNCAA,0
776,2014,146,1246,1276,MNCAA,0
777,2014,152,1163,1196,MNCAA,0
778,2014,152,1246,1458,MNCAA,0


In [4]:
MRegularSeasonDetailedResults = pd.read_csv('Data/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
MRegularSeasonDetailedResults = MRegularSeasonDetailedResults.loc[MRegularSeasonDetailedResults['Season'] >= 2003]
MRegularSeasonDetailedResults = MRegularSeasonDetailedResults[['Season','DayNum','WTeamID','LTeamID']]
MRegularSeasonDetailedResults.loc[:,'Competition'] = 'RegularSeason'
MRegularSeasonDetailedResults.loc[:,'WTeam'] = 0

for i,_ in enumerate(MRegularSeasonDetailedResults.iterrows()):
    if MRegularSeasonDetailedResults.iloc[i,2] > MRegularSeasonDetailedResults.iloc[i,3] :
        MRegularSeasonDetailedResults.iloc[i,2],MRegularSeasonDetailedResults.iloc[i,3] = MRegularSeasonDetailedResults.iloc[i,3],MRegularSeasonDetailedResults.iloc[i,2]
        MRegularSeasonDetailedResults.iloc[i,5] = 1
        
MRegularSeasonDetailedResults = MRegularSeasonDetailedResults.rename(columns={"WTeamID": "Team0_ID", "LTeamID": "Team1_ID"})

MRegularSeasonDetailedResults

,Season,DayNum,Team0_ID,Team1_ID,Competition,WTeam
0,2003,10,1104,1328,RegularSeason,0
1,2003,10,1272,1393,RegularSeason,0
2,2003,11,1266,1437,RegularSeason,0
3,2003,11,1296,1457,RegularSeason,0
4,2003,11,1208,1400,RegularSeason,1
...,...,...,...,...,...,...
87499,2019,132,1153,1222,RegularSeason,0
87500,2019,132,1209,1426,RegularSeason,0
87501,2019,132,1276,1277,RegularSeason,1
87502,2019,132,1382,1387,RegularSeason,1


In [133]:
df = pd.concat([MNCAATourneyDetailedResults,MRegularSeasonDetailedResults])
df

,Season,DayNum,Team0_ID,Team1_ID,Competition,WTeam
0,2003,134,1411,1421,MNCAA,1
1,2003,136,1112,1436,MNCAA,0
2,2003,136,1113,1272,MNCAA,0
3,2003,136,1141,1166,MNCAA,0
4,2003,136,1143,1301,MNCAA,0
...,...,...,...,...,...,...
87499,2019,132,1153,1222,RegularSeason,0
87500,2019,132,1209,1426,RegularSeason,0
87501,2019,132,1276,1277,RegularSeason,1
87502,2019,132,1382,1387,RegularSeason,1


## Ajout de features liées aux Seeds et Conférences

In [197]:
tourney_seed = pd.read_csv('Data/MDataFiles_Stage1/MNCAATourneySeeds.csv')

In [198]:
tourney_result = pd.merge(df, tourney_seed, left_on=['Season', 'Team0_ID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Seed':'SeedT1'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result = pd.merge(tourney_result, tourney_seed, left_on=['Season', 'Team1_ID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Seed':'SeedT2'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)

In [199]:
tourney_result=tourney_result.dropna()

In [200]:
def get_seed(x):
    return str(x[0])

tourney_result['ConfT1'] = tourney_result['SeedT1'].map(lambda x: get_seed(x))
tourney_result['ConfT2'] = tourney_result['SeedT2'].map(lambda x: get_seed(x))


In [201]:
def get_seed(x):
    return int(x[1:3])

tourney_result['SeedT1'] = tourney_result['SeedT1'].map(lambda x: get_seed(x))
tourney_result['SeedT2'] = tourney_result['SeedT2'].map(lambda x: get_seed(x))


In [202]:
tourney_result['Seed_diff'] = tourney_result['SeedT1'] - tourney_result['SeedT2']

In [203]:
tourney_result = tourney_result.reset_index()

In [204]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [205]:
from sklearn.preprocessing import LabelBinarizer

jobs_encoder = LabelBinarizer()
jobs_encoder.fit(tourney_result['ConfT1'])
transformed = jobs_encoder.transform(tourney_result['ConfT1'])
ohe_df = pd.DataFrame(transformed)
tourney_result = pd.concat([tourney_result, ohe_df], axis=1)

In [209]:
tourney_result.rename(columns={0:'W1',1:'X1',2:'Z1',3:'Y1'}, inplace=True)

In [212]:
tourney_result = tourney_result.drop('ConfT1', axis=1)

In [213]:
jobs_encoder = LabelBinarizer()
jobs_encoder.fit(tourney_result['ConfT2'])
transformed = jobs_encoder.transform(tourney_result['ConfT2'])
ohe_df = pd.DataFrame(transformed)
tourney_result = pd.concat([tourney_result, ohe_df], axis=1)

In [214]:
tourney_result.rename(columns={0:'W2',1:'X2',2:'Z2',3:'Y2'}, inplace=True)

In [216]:
tourney_result=tourney_result.drop('ConfT2', axis=1)